# Predicting Heart Disease

## Score: .95326

In [31]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

DATA_DIR = Path("playground-series-s6e2")
OUTPUT_DIR = Path(".")

In [32]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")
original = pd.read_csv(Path("original-data") / "Heart_Disease_Prediction.csv")
np.random.seed(42)
n_copies = 50
noise_scale = 1.5
aug_rows = []
for _, row in original.iterrows():
    for _ in range(n_copies):
        new_row = row.copy()
        for col in original.columns:
            if col == "Heart Disease":
                continue
            new_row[col] = row[col] + np.random.normal(0, noise_scale)
        aug_rows.append(new_row)
aug = pd.DataFrame(aug_rows)
feature_cols_aug = [c for c in aug.columns if c != "Heart Disease"]
for col in feature_cols_aug:
    aug[col] = np.clip(aug[col].round(), 0, 500)
aug["id"] = -1
train = pd.concat([train, aug], ignore_index=True)
print(f"Train (synthetic + {len(aug)} augmented from original): {train.shape}")
print(f"Test: {test.shape}")
train.head()

Train (synthetic + 13500 augmented from original): (643500, 15)
Test: (270000, 14)


,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58.0,1.0,4.0,152.0,239.0,0.0,0.0,158.0,1.0,3.6,2.0,2.0,7.0,Presence
1,1,52.0,1.0,1.0,125.0,325.0,0.0,2.0,171.0,0.0,0.0,1.0,0.0,3.0,Absence
2,2,56.0,0.0,2.0,160.0,188.0,0.0,2.0,151.0,0.0,0.0,1.0,0.0,3.0,Absence
3,3,44.0,0.0,3.0,134.0,229.0,0.0,2.0,150.0,0.0,1.0,2.0,0.0,3.0,Absence
4,4,58.0,1.0,4.0,140.0,234.0,0.0,2.0,125.0,1.0,3.8,2.0,3.0,3.0,Presence


In [33]:
target_col = "Heart Disease"
id_col = "id"
feature_cols = [c for c in train.columns if c not in (id_col, target_col)]

le = LabelEncoder()
y = le.fit_transform(train[target_col])

X_train = train[feature_cols].copy()
X_test = test[feature_cols].copy()

for col in feature_cols:
    if X_train[col].isna().any() or X_test[col].isna().any():
        med = X_train[col].median()
        X_train[col] = X_train[col].fillna(med)
        X_test[col] = X_test[col].fillna(med)

print(f"Features: {feature_cols}")
print(f"Target distribution: {pd.Series(y).value_counts().to_dict()}")

Features: ['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR', 'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium']
Target distribution: {0: 355046, 1: 288454}


In [34]:
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=5,
    min_child_weight=3,
    learning_rate=0.05,
    subsample=0.7,
    colsample_bytree=0.7,
    random_state=42,
    eval_metric="auc",
)

oof_proba = cross_val_predict(model, X_train, y, cv=cv, method="predict_proba")[:, 1]
cv_auc = roc_auc_score(y, oof_proba)
print(f"CV ROC AUC: {cv_auc:.5f}")

CV ROC AUC: 0.95458


In [35]:
seeds = [42, 43, 44, 45, 46]
test_proba_list = []
for seed in seeds:
    m = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=5,
        min_child_weight=3,
        learning_rate=0.05,
        subsample=0.7,
        colsample_bytree=0.7,
        random_state=seed,
        eval_metric="auc",
    )
    m.fit(X_train, y)
    test_proba_list.append(m.predict_proba(X_test)[:, 1])
test_proba = np.mean(test_proba_list, axis=0)

In [36]:
sub = pd.DataFrame({"id": test["id"], "Heart Disease": test_proba})
sub.to_csv(OUTPUT_DIR / "submission.csv", index=False)
print(f"Submission saved to {OUTPUT_DIR / 'submission.csv'}")
sub.head(10)

Submission saved to submission.csv


,id,Heart Disease
0,630000,0.951214
1,630001,0.008201
2,630002,0.983362
3,630003,0.005864
4,630004,0.184621
5,630005,0.980781
6,630006,0.006596
7,630007,0.657193
8,630008,0.990900
9,630009,0.013995
